In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import pandas as pd
from hyperopt.pyll import scope
import time
from sklearn.model_selection import cross_validate, ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor,GradientBoostingRegressor,BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import ExtraTreesRegressor
import math
import gc
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
import nltk 
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import train_test_split,cross_val_predict
import random
import matplotlib.pyplot as plt
import seaborn as sns
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
plt.rcParams["figure.figsize"] = (15,5)
import numpy as np 
import pandas as pd 
from xgboost import plot_importance
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 

In [ ]:
merged=pd.read_csv("drive/MyDrive/Datasets-Collab/merged.csv")

In [ ]:
merged.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
merged.columns[merged.columns.duplicated()]
merged.rename(columns = lambda x: x.replace(' ', '_'), inplace=True)
slice_= 20999

In [ ]:
train=merged.iloc[:slice_,:]
X_test_final=merged.iloc[slice_:,:]
X=train.drop(['Per_Person_Price'],axis=1)
y=train['Per_Person_Price']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(train)
X_test = scaler.transform(X_test_final)

In [ ]:
train=pd.DataFrame(X, index=train.index, columns=train.columns)
X_test_final=pd.DataFrame(X_test, index=X_test_final.index, columns=X_test_final.columns)

In [ ]:
def rmsle(y_pred, data):

    y_true = np.array(data.get_label())
    score = np.sqrt(np.mean(np.power(np.log1p(y_true) - np.log1p(y_pred), 2)))

    return 'rmsle', score, False


In [ ]:

def cv_results(model, X_train, y_train):
  cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
  results = cross_val_score(model, X_train, y_train, cv=cv, n_jobs=-1, scoring='neg_mean_squared_log_error')
  rmse = np.sqrt(-results).mean()
  return rmse

In [ ]:
# List of models to evaluate
models = [
          # Linear regression
          # LinearRegression(),
          # Ridge(),
          # SGDRegressor(),
          # Lasso(),

          # Ensemble methods
          RandomForestRegressor(),
          AdaBoostRegressor(),
          ExtraTreesRegressor(),

          GradientBoostingRegressor(),
          BaggingRegressor(),
          # KNN
          KNeighborsRegressor(),

          # Decision Trees
          lgb.LGBMRegressor(),
          CatBoostRegressor(),

          #XGBoost
          XGBRegressor()
]

# DataFrame to compile results
model_columns = ['model_name', 'rmse', 'time']
base_models = pd.DataFrame(columns=model_columns)

# Populate dataframe with results of each base model
model_index = 0

for model in models:
  start = time.time()
  # Saving model name and paramters
  model_name = model.__class__.__name__
  base_models.loc[model_index, 'model_name'] = model_name

  # Cross validation score
  results = cv_results(model, X, y)
  base_models.loc[model_index, 'rmse'] = results

  #Computation time
  base_models.loc[model_index, 'time'] = time.time() - start

  model_index += 1

base_models = base_models.sort_values(by='rmse', ascending=True)
print(base_models)

                  model_name      rmse      time
7          CatBoostRegressor  0.187771    70.927
6              LGBMRegressor   0.19756   5.04419
8               XGBRegressor  0.244927   39.3601
0      RandomForestRegressor       NaN   1.64852
1          AdaBoostRegressor       NaN  0.462781
2        ExtraTreesRegressor       NaN  0.463525
3  GradientBoostingRegressor       NaN  0.482944
4           BaggingRegressor       NaN   0.53576
5        KNeighborsRegressor       NaN  0.468976


In [ ]:
param_hyperopt = {
    'n_estimators' : scope.int(hp.quniform('n_estimators', 100, 1000, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 2, 20, 1)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1)),
    'subsample': hp.uniform('subsample', 0.8, 1.0),
    'colsample_bytree' : hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'alpha' : hp.choice('alpha', np.arange(0.0, 1.1, 0.1)),
    'booster': hp.choice('booster', ['gbtree']),
    'objective':rmsle
}

In [ ]:
def rmsle(y_pred, data):

    y_true = np.array(data.get_label())
    score = np.sqrt(np.mean(np.power(np.log1p(y_true) - np.log1p(y_pred), 2)))

    return 'rmsle', score, False


In [ ]:
def hyperopt(param_space, X_train_data, y_train_data, num_eval):

    # Timing
    start = time.time()

    # Creating an objective function that will output rmse loss given a set of model parameters
    # Hyperopt will seek to minimize the loss returned by this function
    def objective_function(params):
        clf = XGBRegressor(**params)
        score = cross_val_score(clf, X_train_data, y_train_data, cv=5, scoring='neg_mean_squared_log_error')
        return {'loss':np.sqrt(-score).mean(), 'status':STATUS_OK}

    # Trials object will store information for each trial, allowing us to see under the hood
    trials = Trials()

    # The fmin function will carry out the hyperopt optimization for us
    # Using TPE and given the objective function, the algorithm will find the hyperparameters that minimize loss as defined by our objective function
    best_param = fmin(objective_function,
                      param_space,
                      algo=tpe.suggest,
                      max_evals=num_eval,
                      trials=trials,
                      rstate= np.random.RandomState(0))

    # Loss for each trial
    loss = [x['result']['loss'] for x in trials.trials]

    # Extract our best parameter values into a list
    best_param_values = [x for x in best_param.values()]

    # Fit a new model based on the best parameters, which are stored in the list alphabetically    
    clf_best = XGBRegressor(
        alpha = best_param_values[0],
        booster = 'gbtree',
        colsample_bytree = best_param_values[2],
        learning_rate = best_param_values[3],
        max_depth = int(best_param_values[4]),
        n_estimators = int(best_param_values[5]),
        subsample = best_param_values[6]
          )

    clf_best.fit(X_train_data, y_train_data,eval_metric=rmsle,verbose=True)

    print("")
    print("##### Results")
    print("Score best parameters: ", min(loss))
    print("Best parameters: ", best_param)
    print("Time elapsed: ", time.time() - start)
    print("Parameter combinations evaluated: ", num_eval)

    return trials, clf_best

In [ ]:
hyperopt_trials, xgb_hyperopt = hyperopt(
    param_space=param_hyperopt,
    X_train_data=X,
    y_train_data=y,
    num_eval=3)

[20:55:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  0%|          | 0/3 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
AttributeError: 'numpy.ndarray' object has no attribute 'get_label'

  FitFailedWarning)



[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:55:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

AssertionError: ignored

In [ ]:
def cv_results_print(model, X_train, y_train):
  start = time.time()
  cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
  results = cross_val_score(model, X_train, y_train, cv=cv, n_jobs=-1, scoring='neg_mean_squared_log_error')
  rmse = np.sqrt(-results).mean()

  model_name = model.__class__.__name__
  model_params = model.get_params()
  print('Model: {}'.format(model_name))
  print('Model Parameters: {}'.format(model_params))
  print('5 Fold CV RMSE: {:.4f}'.format(rmse))
  print('Computation Time: {:.2f}'.format(time.time() - start))

In [ ]:
cv_results_print(xgb_hyperopt, X, y)

TypeError: ignored

In [ ]:
from xgboost import plot_importance

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
   
  if useTrainCV: 
    xgb_param = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
        metrics='rmse', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
    
    alg.set_params(n_estimators=cvresult.shape[0])

  print(cvresult.shape[0])
  #Fit the algorithm on the data
  alg.fit(dtrain[predictors], dtrain['Per_Person_Price'],eval_metric='rmse')
      
  #Predict training set:
  dtrain_predictions = alg.predict(dtrain[predictors])
  #dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
      
  #Print model report:
  print ("\nModel Report")
  print ("RMSLE : %.4g" %  np.sqrt(mean_squared_log_error(dtrain['Per_Person_Price'].values, dtrain_predictions)))
  #print "RMSLE (Train): %f" % metrics.roc_auc_score(dtrain['Per_Person_Price'], dtrain_predprob)
  try:        
    fig, ax = plt.subplots(1,1,figsize=figsize)
    plot_importance(booster=alg, ax=ax)   
  except:
    pass

In [ ]:
target='Per_Person_Price'

predictors = [x for x in X_train.columns if x not in [target]]

xgb1 = XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=8,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:squarederror',
 nthread=4,
 scale_pos_weight=1,
 seed=27)


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_test1 = {
 'max_depth':range(3,15,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=437, max_depth=8,
 min_child_weight=1, gamma=0, subsample=0.9, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 

param_grid = param_test1, scoring='neg_root_mean_squared_error',n_jobs=4,iid=False,cv=4)


gsearch1.fit(X_train[predictors],X_train[target])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([137.10920393, 137.97056758, 138.33289719, 208.55285215,
         208.39987266, 211.57123601, 288.26888865, 283.92457569,
         281.33597028, 380.31921905, 361.47836912, 360.65028852,
         462.68976325, 439.525267  , 434.39804047, 553.80408853,
         527.25888914, 493.33392453]),
  'mean_score_time': array([0.31381148, 0.31409019, 0.33518362, 0.40304309, 0.37552756,
         0.36205733, 0.60620612, 0.51523685, 0.51001781, 0.86455441,
         0.73442781, 0.74242169, 1.19096702, 1.02359849, 0.97774619,
         1.66721511, 1.33466268, 0.78774822]),
  'mean_test_score': array([-4764.44331402, -4806.8197904 , -4814.36370252, -4470.45754624,
         -4490.25280263, -4524.89958673, -4405.49669037, -4428.04378325,
         -4403.92243238, -4409.34267015, -4405.6476496 , -4392.38900891,
         -4449.97103751, -4436.50017966, -4434.24069188, -4512.9981788 ,
         -4485.37031754, -4437.70053877]),
  'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5,

In [ ]:
param_test1 = {
 'max_depth':[8,9,10],
 'min_child_weight':[4,5,6]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=437, max_depth=8,
 min_child_weight=1, gamma=0, subsample=0.9, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 

param_grid = param_test1, scoring='neg_root_mean_squared_error',n_jobs=4,iid=False,cv=4)


gsearch1.fit(X_train[predictors],X_train[target])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([327.28820622, 324.97725344, 321.99091452, 354.25844616,
         350.1393314 , 351.57459354, 400.93035245, 396.01267719,
         384.17626458]),
  'mean_score_time': array([0.53357178, 0.51312691, 0.48718965, 0.57661521, 0.57032239,
         0.68233627, 0.82114893, 0.81224978, 0.4477666 ]),
  'mean_test_score': array([-4383.17534107, -4399.82354965, -4419.85679372, -4395.97027111,
         -4392.38900891, -4433.9316791 , -4409.75818437, -4398.67150846,
         -4437.58474944]),
  'param_max_depth': masked_array(data=[8, 8, 8, 9, 9, 9, 10, 10, 10],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_min_child_weight': masked_array(data=[4, 5, 6, 4, 5, 6, 4, 5, 6],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'params': [{'max_dept

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=437, max_depth=8,
 min_child_weight=4, gamma=0, subsample=0.9, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 

param_grid = param_test3, scoring='neg_root_mean_squared_error',n_jobs=4,iid=False,cv=4)

gsearch1.fit(X_train[predictors],X_train[target])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([328.86963403, 324.19549239, 328.46230924, 328.02756613,
         323.55264866]),
  'mean_score_time': array([0.6145137 , 0.55590796, 0.58160603, 0.55762368, 0.33785737]),
  'mean_test_score': array([-4383.17534107, -4383.17534107, -4383.17534107, -4383.17534107,
         -4383.17534107]),
  'param_gamma': masked_array(data=[0.0, 0.1, 0.2, 0.3, 0.4],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.1},
   {'gamma': 0.2},
   {'gamma': 0.3},
   {'gamma': 0.4}],
  'rank_test_score': array([1, 1, 1, 1, 1], dtype=int32),
  'split0_test_score': array([-4409.44984035, -4409.44984035, -4409.44984035, -4409.44984035,
         -4409.44984035]),
  'split1_test_score': array([-4064.80784059, -4064.80784059, -4064.80784059, -4064.80784059,
         -4064.80784059]),
  'split2_test_score': array([-4204.74758916, -4204.74758916, -4204.74758916, -4204.74758916,
         -4

In [ ]:
target='Per_Person_Price'

predictors = [x for x in X_train.columns if x not in [target]]

xgb2 = XGBRegressor(
 learning_rate =0.1,
 n_estimators=2000,
 max_depth=8,
 min_child_weight=4,
 gamma=0,
 subsample=0.9,
 colsample_bytree=0.8,
 objective= 'reg:squarederror',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb2, X_train, predictors)

518

Model Report
RMSLE : 0.1166


In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gsearch4 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=518, max_depth=8,
 min_child_weight=4, gamma=0, subsample=0.9, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='neg_root_mean_squared_error',n_jobs=4,iid=False, cv=5)

gsearch4.fit(X_train[predictors],X_train[target])


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([386.53951235, 363.042735  , 338.37822924, 313.3070035 ,
         433.91691971, 417.32392507, 392.03924212, 362.10139155,
         495.66273103, 469.06437483, 432.19817591, 396.51340113,
         539.60126262, 514.24307046, 475.9884151 , 431.9227118 ]),
  'mean_score_time': array([0.51508894, 0.49074626, 0.55725942, 0.50304551, 0.50650969,
         0.54222612, 0.51850076, 0.53687315, 0.53671861, 0.55969405,
         0.5549365 , 0.55097036, 0.54518681, 0.52166924, 0.61391954,
         0.33001447]),
  'mean_test_score': array([-4445.90781459, -4460.59786581, -4397.04382855, -4387.32062126,
         -4448.06398177, -4450.15904614, -4415.20845629, -4416.59335342,
         -4450.08908655, -4443.18480748, -4436.98396207, -4427.88942775,
         -4459.39004002, -4447.95111081, -4447.4044389 , -4436.5845876 ]),
  'param_colsample_bytree': masked_array(data=[0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8,
                     0.8, 0.9, 0.9, 0.9, 0.9],
          

In [ ]:
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

({'mean_fit_time': array([386.53951235, 363.042735  , 338.37822924, 313.3070035 ,
         433.91691971, 417.32392507, 392.03924212, 362.10139155,
         495.66273103, 469.06437483, 432.19817591, 396.51340113,
         539.60126262, 514.24307046, 475.9884151 , 431.9227118 ]),
  'mean_score_time': array([0.51508894, 0.49074626, 0.55725942, 0.50304551, 0.50650969,
         0.54222612, 0.51850076, 0.53687315, 0.53671861, 0.55969405,
         0.5549365 , 0.55097036, 0.54518681, 0.52166924, 0.61391954,
         0.33001447]),
  'mean_test_score': array([-4445.90781459, -4460.59786581, -4397.04382855, -4387.32062126,
         -4448.06398177, -4450.15904614, -4415.20845629, -4416.59335342,
         -4450.08908655, -4443.18480748, -4436.98396207, -4427.88942775,
         -4459.39004002, -4447.95111081, -4447.4044389 , -4436.5845876 ]),
  'param_colsample_bytree': masked_array(data=[0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8,
                     0.8, 0.9, 0.9, 0.9, 0.9],
          